<a href="https://colab.research.google.com/github/Mk-lab-sudo/Sakhi-Phase1/blob/main/Sakhi_Framework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
!pip install rasa
!pip install datasets


In [ ]:
from transformers import pipeline


In [ ]:
# Load the GPT-Neo model
generator = pipeline("text-generation", model="EleutherAI/gpt-neo-1.3B")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.35k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/5.31G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

Device set to use cpu


In [ ]:
# Test Sakhi with a prompt
prompt = "Hello, I am Sakhi, your AI assistant. How can I assist you today?"
result = generator(prompt, max_length=50, num_return_sequences=1)
print(result[0]['generated_text'])


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Hello, I am Sakhi, your AI assistant. How can I assist you today?

A: Hello, AI!

Q: How can I assist you today?

A: You can try the following methods.




In [ ]:
from datasets import load_dataset

# Use an alternative conversational dataset
dataset = load_dataset("daily_dialog", split="train")

# Preview the first entry
print(dataset[0])


README.md:   0%|          | 0.00/7.27k [00:00<?, ?B/s]

daily_dialog.py:   0%|          | 0.00/4.85k [00:00<?, ?B/s]

The repository for daily_dialog contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/daily_dialog.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split:   0%|          | 0/11118 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1000 [00:00<?, ? examples/s]

{'dialog': ['Say , Jim , how about going for a few beers after dinner ? ', ' You know that is tempting but is really not good for our fitness . ', ' What do you mean ? It will help us to relax . ', " Do you really think so ? I don't . It will just make us fat and act silly . Remember last time ? ", " I guess you are right.But what shall we do ? I don't feel like sitting at home . ", ' I suggest a walk over to the gym where we can play singsong and meet some of our friends . ', " That's a good idea . I hear Mary and Sally often go there to play pingpong.Perhaps we can make a foursome with them . ", ' Sounds great to me ! If they are willing , we could ask them to go dancing with us.That is excellent exercise and fun , too . ', " Good.Let ' s go now . ", ' All right . '], 'act': [3, 4, 2, 2, 2, 3, 4, 1, 3, 4], 'emotion': [0, 0, 0, 0, 0, 0, 4, 4, 4, 4]}


In [ ]:
# Save the generator model
generator.save_pretrained("sakhi_nlp_model")


In [ ]:
from datasets import load_dataset

# Load the dataset (e.g., daily_dialog)
dataset = load_dataset("daily_dialog", split="train")


In [ ]:
def format_data(sample):
    return {
        "prompt": sample["dialog"][0],
        "response": sample["dialog"][1] if len(sample["dialog"]) > 1 else ""
    }


In [ ]:
formatted_data = dataset.map(format_data)


Map:   0%|          | 0/11118 [00:00<?, ? examples/s]

In [ ]:
import json

# Convert to a list of dictionaries and save to a JSON file
formatted_data_list = [item for item in formatted_data]
with open("daily_dialog_formatted.json", "w") as f:
    json.dump(formatted_data_list, f, indent=4)


In [ ]:
print(type(formatted_data))  # Should print: <class 'datasets.Dataset'>
print(len(formatted_data))   # Number of entries
print(formatted_data[0])     # Check the first formatted entry


<class 'datasets.arrow_dataset.Dataset'>
11118
{'dialog': ['Say , Jim , how about going for a few beers after dinner ? ', ' You know that is tempting but is really not good for our fitness . ', ' What do you mean ? It will help us to relax . ', " Do you really think so ? I don't . It will just make us fat and act silly . Remember last time ? ", " I guess you are right.But what shall we do ? I don't feel like sitting at home . ", ' I suggest a walk over to the gym where we can play singsong and meet some of our friends . ', " That's a good idea . I hear Mary and Sally often go there to play pingpong.Perhaps we can make a foursome with them . ", ' Sounds great to me ! If they are willing , we could ask them to go dancing with us.That is excellent exercise and fun , too . ', " Good.Let ' s go now . ", ' All right . '], 'act': [3, 4, 2, 2, 2, 3, 4, 1, 3, 4], 'emotion': [0, 0, 0, 0, 0, 0, 4, 4, 4, 4], 'prompt': 'Say , Jim , how about going for a few beers after dinner ? ', 'response': ' You

In [ ]:
# Save the generator model
generator.save_pretrained("sakhi_nlp_model")


In [ ]:
test_prompts = [
    "What is your name?",
    "Can you help me plan my day?",
    "Tell me a joke!",
]

for prompt in test_prompts:
    result = generator(prompt, max_length=50, num_return_sequences=1)
    print(f"Prompt: {prompt}")
    print(f"Response: {result[0]['generated_text']}\n")


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Prompt: What is your name?
Response: What is your name?

A:

My name is 

Szabo Zsambok

I also have the following two entries in dbo.tblSales where your Szabo is located:-
s



Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Prompt: Can you help me plan my day?
Response: Can you help me plan my day?

Is there someone I should know about?

Do you think my life needs some form of attention?

This is the question I come to most often, but I have also received it in

Prompt: Tell me a joke!
Response: Tell me a joke!

I am a writer and I love writing. But even I couldn’t write an autobiography without a few funny lines. This is a story of my life as a writer, that I hope can be enjoyable for



In [ ]:
!pip install transformers datasets torch accelerate


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# Load a pre-trained model and tokenizer
model_name = "EleutherAI/gpt-neo-125M"  # or "gpt2"
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)


config.json:   0%|          | 0.00/1.01k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/526M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/357 [00:00<?, ?B/s]

In [ ]:
from datasets import Dataset

# Load your formatted dataset
import json
with open("daily_dialog_formatted.json", "r") as f:
    data = json.load(f)

# Convert to a Hugging Face Dataset
dataset = Dataset.from_list(data)

# Assign padding token
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Define tokenization function
def tokenize_data(sample):
    input_text = f"Prompt: {sample['prompt']}\nResponse: {sample['response']}\n"
    tokenized = tokenizer(input_text, truncation=True, padding="max_length", max_length=1000)
    return {
        "input_ids": tokenized["input_ids"],
        "attention_mask": tokenized["attention_mask"]
    }

# Apply tokenization with adjusted schema
tokenized_dataset = dataset.map(tokenize_data, batched=True, remove_columns=dataset.column_names)


Map:   0%|          | 0/11118 [00:00<?, ? examples/s]

In [ ]:
from transformers import DataCollatorWithPadding

# Define the data collator
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)


In [ ]:
from datasets import Dataset

# Load your formatted dataset
import json
with open("daily_dialog_formatted.json", "r") as f:
    data = json.load(f)

# Convert to a Hugging Face Dataset
dataset = Dataset.from_list(data)


# Assign padding token
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Define tokenization function
def tokenize_data(sample):
    input_text = f"Prompt: {sample['prompt']}\nResponse: {sample['response']}\n"
    tokenized = tokenizer(input_text, truncation=True, padding="max_length", max_length=1000, return_tensors="pt")
    return {
        "input_ids": tokenized["input_ids"],
        "attention_mask": tokenized["attention_mask"]
    }

# Apply tokenization with adjusted schema
tokenized_dataset = dataset.map(tokenize_data, batched=True, remove_columns=dataset.column_names)


Map:   0%|          | 0/11118 [00:00<?, ? examples/s]

In [ ]:
from transformers import Trainer, TrainingArguments, AutoModelForCausalLM, AutoTokenizer

# Load a pre-trained model and tokenizer
model_name = "EleutherAI/gpt-neo-125M"  # or "gpt2"
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

from datasets import Dataset

# Load your formatted dataset
import json
with open("daily_dialog_formatted.json", "r") as f:
    data = json.load(f)

# Convert to a Hugging Face Dataset
dataset = Dataset.from_list(data)

# Assign padding token
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Define tokenization function
def tokenize_data(sample):
    input_text = f"Prompt: {sample['prompt']}\nResponse: {sample['response']}\n"
    tokenized = tokenizer(input_text, truncation=True, padding="max_length", max_length=1000, return_tensors="pt")
    # Add labels - the input_ids shifted to the right are the labels
    tokenized["labels"] = tokenized["input_ids"].clone()
    return tokenized

# Apply tokenization with adjusted schema
tokenized_dataset = dataset.map(tokenize_data, batched=True, remove_columns=dataset.column_names)

from transformers import TrainingArguments, Trainer

# Define training arguments
training_args = TrainingArguments(
    output_dir="./sakhi_model",
    evaluation_strategy="no",
    learning_rate=5e-5,
    per_device_train_batch_size=4,
    num_train_epochs=3,
    save_steps=500,
    save_total_limit=2,
    logging_dir="./logs",
    logging_steps=2,
    logging_first_step=True,
)

import os
# Disable wandb if not needed
os.environ["WANDB_DISABLED"] = "true"

# Define the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
)



Map:   0%|          | 0/11118 [00:00<?, ? examples/s]

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
<ipython-input-2-f4ae36240d92>:54: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
# Train the model
trainer.train()

# Save the fine-tuned model
trainer.save_model("./sakhi_fine_tuned")
tokenizer.save_pretrained("./sakhi_fine_tuned")

Step,Training Loss


In [ ]:
pip install wandb